# Span: Spectral Analysis

#### Span is a set of tools to fit spectra the of binary stars simultaneously.

#### This means that we need synthetic atmosphere models to do the fit. In this example we use [TLUSTY](http://tlusty.oca.eu) models for stars hotter than 15000 K, and ATLAS 9 models (from [Howarth 2011](https://ui.adsabs.harvard.edu/abs/2011MNRAS.413.1515H/abstract)) for temperatures 10000-15000 K, but you can use your favourite models.

#### If you use any of these models in your work (available at minato>models) please use the correct reference:

#### - ATLAS 9 late B-type stars grid: [Howarth 2011](https://ui.adsabs.harvard.edu/abs/2011MNRAS.413.1515H/abstract)
#### - TLUSTY B-type stars grid: [Lanz & Hubeny 2007](https://ui.adsabs.harvard.edu/abs/2007ApJS..169...83L/abstract)
#### - TLUSTY O-type stars grid: [Lanz & Hubeny 2003](https://ui.adsabs.harvard.edu/abs/2003ApJS..146..417L/abstract)

Let's start by creating a simple grid of parameters. Parameters currently accepted by Span are: effective temperature (both stars) surface gravity (both stars), rotational velocity (both stars), the light ratio (defined as the contribution of the secondary star), and the Helium enrichment (primary star).

Here we'll create a grid similar to the one of [Villaseñor+2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230707766V/abstract) but less dense. How dense and broad is your grid will depend on the information you have on your binary. Note that you can do this using `numpy`, `range`, or a simple list `[]`

This grid should run for about 20 minutes, so to make it faster for this example let's fix the rotational velocity of the companion (rotB). It should now be just a couple of minutes.

In [1]:
import numpy as np

teffA = np.linspace(10, 15, 6, dtype=int)
loggA = [20, 22, 24, 26, 28]
rotA = list(range(20, 70, 10))
teffB = [22, 24, 26, 28, 30, 32.5]
loggB = [25, 30, 35, 40]
# rotB = list(range(0, 600, 100))
rotB = [200]
lrat = np.linspace(30, 45, 4)/100
he2h = np.linspace(8, 12, 5)/100

pars_list = [lrat, he2h, teffA, loggA, rotA, teffB, loggB, rotB]

gridlist = [list(l) for l in pars_list]
for par in gridlist:
    print(par)

[0.3, 0.35, 0.4, 0.45]
[0.08, 0.09, 0.1, 0.11, 0.12]
[10, 11, 12, 13, 14, 15]
[20, 22, 24, 26, 28]
[20, 30, 40, 50, 60]
[22, 24, 26, 28, 30, 32.5]
[25, 30, 35, 40]
[200]


>
>#### **Note:** the light ratio is defined as flux_secondary / total_flux
>

In the next step we need to specify our observed spectra. In our example we're using the disentangled spectra from [Villaseñor+2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230707766V/abstract).

And also we have to specify what spectral lines we want to be considered in the fit

In [2]:
dsnt_A = './example_spectra/ADIS_lguess_K1K2=0.3_94.0_15.0.txt'
dsnt_B = './example_spectra/BDIS_lguess_K1K2=0.3_94.0_15.0.txt'

select_linesA = [4026, 4102, 4121, 4144, 4267, 4340, 4388, 4471, 4553]
select_linesB = [4026, 4102, 4121, 4144, 4267, 4340, 4388, 4553]

We will now use `span.atmfit` to create an object containing all the previous information and we'll use `compute_chi2` to do the actual fit

In [3]:
from minato import span as sp

fit_obj = sp.atmfit(gridlist, dsnt_A, dsnt_B, lrat0=0.3)
df_results = fit_obj.compute_chi2(select_linesA, select_linesB)
display(df_results)


 ##### Computing chi^2 for grid #####


#     Warning: you are using an initial light ratio of 0.3 



ValueError: operands could not be broadcast together with shapes (1150,) (1180,) 

Now that we have our dataframe with the results, we should save it to work with it later if we want to. Since this can get very heavy, I recommend to use the feather format

In [4]:
df_results.to_feather('atmfit_results.feather')

See the tutorial "__Working with atmfit results__" to see how to get the best-fitting parameters